##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pipeline TFX simples para Vertex Pipelines


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Executar no Google Cloud Vertex AI Workbench</a></td>
</table></div>


Este tutorial baseado em notebook criará um pipeline TFX simples e o executará usando o Google Cloud Vertex Pipelines. Este notebook é baseado no pipeline TFX que construímos no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). Se você não está familiarizado com o TFX e ainda não leu esse tutorial, leia-o antes de prosseguir com este notebook.

O Google Cloud Vertex Pipelines ajuda você a automatizar, monitorar e controlar seus sistemas de ML orquestrando seu workflow de ML sem servidor. Você pode definir seus pipelines de ML usando Python com TFX e, em seguida, executá-los no Google Cloud. Consulte a [Introdução ao Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) para saber mais sobre o Vertex Pipelines.

Este notebook foi criado para ser executado no [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) ou em [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks). Se você não estiver usando um deles, basta clicar no botão "Executar no Google Colab" acima.

## Configuração

Antes de executar este notebook, certifique-se de ter o seguinte:

- Um projeto [Google Cloud Platform](http://cloud.google.com/).
- Um bucket do [Google Cloud Storage](https://cloud.google.com/storage). Veja o [Guia para criar buckets](https://cloud.google.com/storage/docs/creating-buckets).
- A [Vertex AI e a API Cloud Storage](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage-component.googleapis.com) ativados.

Consulte a [documentação do Vertex](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project) para configurar melhor seu projeto GCP.

### Instale os pacotes Python

Instalaremos os pacotes Python necessários, incluindo TFX e KFP, para criar pipelines de ML e enviar trabalhos para o Vertex Pipelines.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

#### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Se você não estiver no Colab, poderá reiniciar o runtime com a seguinte célula.

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### Faça login no Google para este notebook

Se você estiver executando este notebook no Colab, autentique-se com sua conta de usuário:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**Se você estiver no AI Platform Notebooks**, autentique-se no Google Cloud antes de executar a próxima seção, executando

```sh
gcloud auth login
```

**na janela do Terminal** (que você pode abrir em **File** &gt; **New** no menu). Você só precisa fazer isso uma vez por instância de notebook.

Verifique as versões dos pacotes.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### Configuração de variáveis

Definiremos algumas variáveis ​​usadas para personalizar os pipelines abaixo. As seguintes informações são necessárias:

- ID do projeto GCP. Consulte [Identificando o ID do seu projeto](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).
- Região do GCP para executar pipelines. Para obter mais informações sobre as regiões em que o Vertex Pipelines está disponível, veja o [guia de locais do Vertex AI](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability).
- Bucket do Google Cloud Storage para armazenar saídas de pipeline.

**Insira os valores necessários na célula abaixo antes de executá-la**.


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

Defina o `gcloud` para usar seu projeto.

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-pipelines'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for input data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### Preparação dos dados de exemplo

Usaremos o mesmo [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Existem quatro características numéricas neste dataset que já foram normalizados para ter intervalo [0,1]. Construiremos um modelo de classificação que prevê as `species` de pinguins.

Precisamos fazer nossa própria cópia do dataset. Como o TFX ExampleGen lê entradas de um diretório, precisamos criar um diretório e copiar o dataset para ele no GCS.

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

Dê uma olhada rápida no arquivo CSV.

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## Crie um pipeline

Os pipelines TFX são definidos usando APIs Python. Definiremos um pipeline que consiste em três componentes, CsvExampleGen, Trainer e Pusher. A definição do pipeline e do modelo é quase a mesma do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

A única diferença é que não precisamos definir `metadata_connection_config` que é usado para localizar o banco de dados [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd). Como o Vertex Pipelines usa um serviço gerenciado de metadados, os usuários não precisam se preocupar com isso e não precisamos especificar o parâmetro.

Antes de realmente definir o pipeline, precisamos primeiro escrever um código de modelo para o componente Trainer.

### Escreva o código do modelo.

Usaremos o mesmo código de modelo do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils


from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Copie o arquivo do módulo para o GCS, que pode ser acessado nos componentes do pipeline. Como o treinamento do modelo acontece no GCP, precisamos fazer upload desta definição de modelo.

Caso contrário, talvez você queira criar uma imagem de container incluindo o arquivo do módulo e usar a imagem para executar o pipeline.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### Escreva uma definição de pipeline

Definiremos uma função para criar um pipeline TFX.

In [ ]:
# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified because we don't need `metadata_path` argument.

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     ) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Execute o pipeline no Vertex Pipelines.

Usamos `LocalDagRunner` que roda em ambiente local no [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). O TFX fornece vários orquestradores para executar seu pipeline. Neste tutorial usaremos o Vertex Pipelines junto com o dag runner Kubeflow V2.

Precisamos definir um executor para realmente executar o pipeline. Você compilará seu pipeline em nosso formato de definição de pipeline usando APIs TFX.

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR))

O arquivo de definição gerado pode ser enviado usando o cliente kfp.

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Agora você pode visitar o link na saída acima ou visitar 'Vertex AI &gt; Pipelines' no [Console do Google Cloud](https://console.cloud.google.com/) para acompanhar o andamento.